In [2]:
#肩舉
#0730更新 倒數完會摧毀然後開新的
import cv2
import mediapipe as mp
import numpy as np
import random
import time

def show_countdown(seconds):
    # 初始化摄像头
    cap = cv2.VideoCapture(0)

    # 获取摄像头图像的宽度和高度
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # 定义视频编解码器和输出文件名
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_filename = 'countdown_output.avi'
    out = cv2.VideoWriter(output_filename, fourcc, 20.0, (width, height))
    # 倒计时开始
    start_time = time.time()
    while True:
        ret, frame = cap.read()

        # 计算倒计时数字
        elapsed_time = time.time() - start_time
        remaining_time = max(seconds - int(elapsed_time), 0)

        # 在图像上显示倒计时数字
        font = cv2.FONT_HERSHEY_SIMPLEX
        text = f"Countdown: {remaining_time}"
        cv2.putText(frame, text, (50, 50), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # 在窗口中显示倒计时图像
        cv2.imshow('Countdown', frame)
        out.write(frame)

        # 如果倒计时结束，退出循环
        if remaining_time == 0:
            break

        # 每帧显示时间间隔
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 释放摄像头，关闭窗口
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

countdown_seconds = 5
show_countdown(countdown_seconds)


def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

def mainCourse():
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    cap = cv2.VideoCapture(0)
    # Curl counter variables
    counter = 0 
    stage = None

    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        time.sleep(1)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
            try:

                landmarks = results.pose_landmarks.landmark

                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)

                # Visualize angle
                cv2.putText(image, str(angle), 
                               tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )




                # Curl counter logic

                if angle > 80 and angle < 100: #counter一直跳的原因在於我一開始寫angle>80 但我手伸直也是>80 所以它沒有真的變成down 我就一直維持在up
                    stage = "down"
                    times = 0

                if angle > 160 and angle < 175 and stage =='down' and wrist[1] < shoulder[1]:
                    stage="up"
                    counter +=1
                    print(counter)



            except:
                pass


            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (265,95), (245,117,16), -1)

            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

            # Stage data
            cv2.putText(image, 'STAGE', (95,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (90,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )               



            cv2.imshow('Mediapipe Feed', image)



            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
        
# 等待一段時間，讓攝像頭準備好
time.sleep(1)
mainCourse()